In [18]:
import os
from os import listdir
import pandas as pd
import numpy as np
from datetime import datetime as dt
import itertools

In [19]:
files = []
data_dir = 'D:\Github\Football Analysis\data\\'
filenames = [ filename for filename in listdir(data_dir) if filename.endswith('.csv') ]
print(filenames)

['05.csv', '06.csv', '07.csv', '08.csv', '09.csv', '10.csv', '11.csv', '12.csv', '13.csv', '14.csv', '15.csv', '16.csv', '17.csv', '18.csv', 'final_dataset.csv', 'test.csv']


In [20]:
# root_dir = os.path.abspath('../..')
data_dir = 'D:\Github\Football Analysis\data\\'
data = []
for file in filenames:
    data.append(pd.read_csv(data_dir + file))

# cols = ['HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST','HF','AF','HC','AC','HY','AY','HR','AR']
# data = []
# for ind, file in enumerate(filenames):
#     dat = pd.read_csv(os.path.join(data_dir,file), sep=',')
#     if ind==0:
#         to_be_taken=dat.loc[:,cols]
#         data = to_be_taken
#     else:
#         to_be_taken=dat.loc[:,cols]
#         data = pd.concat([data,to_be_taken])
#     data=data.reset_index(drop=True)    
# teams = np.unique(data.loc[:,'HomeTeam'].tolist())

In [23]:
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%d/%m/%y').date()
    

def parse_date_other(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%d/%m/%Y').date()

play_stats = []
cols_req = ['Date','HomeTeam','AwayTeam','FTHG','FTAG','FTR']
for i in range(0,14):
#     data[i].Date = data[i].Date.apply(parse_date)
    play_stats.append(data[i][cols_req])
# for team in teams:
#     vals=[]
#     for index, row in data.head().iterrows():
#         if index>10:
#             break
#         if row['HomeTeam']==team:
#             vals[0]+=row['FTHG']
#             vals[1]+=row['HTHG']
#             vals[0]+=row['FTHG']
#             vals[0]+=row['FTHG']
#             vals[0]+=row['FTHG']
#         elif row['AwayTeam']==team:
#             print(team)

In [5]:
def goals_scored(play_stat):
    teams = {}
    for i in play_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
    for i in range(len(play_stat)):
        HTGS = play_stat.iloc[i]['FTHG']
        ATGS = play_stat.iloc[i]['FTAG']
        teams[play_stat.iloc[i].HomeTeam].append(HTGS)
        teams[play_stat.iloc[i].AwayTeam].append(ATGS)
        
    goals_s = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    goals_s[0] = 0
    for i in range(2,39):
        goals_s[i] = goals_s[i] + goals_s[i-1]
    return goals_s

def goals_conceded(play_stat):
    teams = {}
    for i in play_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
    for i in range(len(play_stat)):
        HTGC = play_stat.iloc[i]['FTAG']
        ATGC = play_stat.iloc[i]['FTHG']
        teams[play_stat.iloc[i].HomeTeam].append(HTGC)
        teams[play_stat.iloc[i].AwayTeam].append(ATGC)
        
    goals_c = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    goals_c[0] = 0
    for i in range(2,39):
        goals_c[i] = goals_c[i] + goals_c[i-1]
    return goals_c


def get_gss(play_stat):
    GC = goals_conceded(play_stat)
    GS = goals_scored(play_stat)
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = play_stat.iloc[i].HomeTeam
        at = play_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    play_stat['HTGS'] = HTGS
    play_stat['ATGS'] = ATGS
    play_stat['HTGC'] = HTGC
    play_stat['ATGC'] = ATGC
    
    return play_stat

In [6]:
def points(result):
    point = 0
    if result == 'W':
        point = 3
    elif result == 'D':
        point = 1
    return point

def get_cuml_points(res):
    res_points = res.applymap(points)
    for i in range(2,39):
        res_points[i] = res_points[i] + res_points[i-1]
        
    res_points.insert(column =0, loc = 0, value = [0*i for i in range(20)])
    return res_points

def getres(play_stat):
    teams = {}
    for i in play_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
        
    for i in range(len(play_stat)):
        if play_stat.iloc[i].FTR == 'H':
            teams[play_stat.iloc[i].HomeTeam].append('W')
            teams[play_stat.iloc[i].AwayTeam].append('L')
        elif play_stat.iloc[i].FTR == 'A':
            teams[play_stat.iloc[i].HomeTeam].append('L')
            teams[play_stat.iloc[i].AwayTeam].append('W')
        else:
            teams[play_stat.iloc[i].HomeTeam].append('D')
            teams[play_stat.iloc[i].AwayTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

def get_agg_points(play_stat):
    res = getres(play_stat)
    cuml_pts = get_cuml_points(res)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = play_stat.iloc[i].HomeTeam
        at = play_stat.iloc[i].AwayTeam
        HTP.append(cuml_pts.loc[ht][j])
        ATP.append(cuml_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
            
    play_stat['HTP'] = HTP
    play_stat['ATP'] = ATP
    return play_stat

In [7]:
def get_form(play_stat,num):
    form = getres(play_stat)
    form_final = form.copy()
    for i in range(num,39):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1           
    return form_final

def add_form(play_stat,num):
    form = get_form(play_stat,num)
    h = ['M' for i in range(num * 10)] 
    a = ['M' for i in range(num * 10)]
    
    j = num
    for i in range((num*10),380):
        ht = play_stat.iloc[i].HomeTeam
        at = play_stat.iloc[i].AwayTeam
        
        past = form.loc[ht][j]               
        h.append(past[num-1])                    
        
        past = form.loc[at][j]               
        a.append(past[num-1])                
        
        if ((i + 1)% 10) == 0:
            j = j + 1

    play_stat['HM' + str(num)] = h                 
    play_stat['AM' + str(num)] = a

    
    return play_stat


def add_form_df(play_stats):
    for j in range(1,6):
        play_stats = add_form(play_stats,j)
    return play_stats

In [8]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]

In [9]:
def get_mw(play_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    play_stat['MW'] = MatchWeek
    return play_stat

In [10]:
for i in range(len(play_stats)):
    play_stats[i] = get_gss(play_stats[i])
    play_stats[i] = get_agg_points(play_stats[i])
    play_stats[i] = add_form_df(play_stats[i])
    play_stats[i] = play_stats[i][cols]
    play_stats[i] = get_mw(play_stats[i])

C:\Users\jimit\Anaconda3\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jimit\Anaconda3\lib\site-packages\ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\jimit\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

In [11]:
play_stat = play_stats[0]
for i in range(1,len(play_stats)):
    play_stat = pd.concat([play_stat,play_stats[i]], ignore_index=True)


# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += points(letter)
    return sum

play_stat['HTFormPtsStr'] = play_stat['HM1'] + play_stat['HM2'] + play_stat['HM3'] + play_stat['HM4'] + play_stat['HM5']
play_stat['ATFormPtsStr'] = play_stat['AM1'] + play_stat['AM2'] + play_stat['AM3'] + play_stat['AM4'] + play_stat['AM5']

play_stat['HTFormPts'] = play_stat['HTFormPtsStr'].apply(get_form_points)
play_stat['ATFormPts'] = play_stat['ATFormPtsStr'].apply(get_form_points)

# Identify Win/Loss Streaks if any.
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0
    
play_stat['HTWinStreak3'] = play_stat['HTFormPtsStr'].apply(get_3game_ws)
play_stat['HTWinStreak5'] = play_stat['HTFormPtsStr'].apply(get_5game_ws)
play_stat['HTLossStreak3'] = play_stat['HTFormPtsStr'].apply(get_3game_ls)
play_stat['HTLossStreak5'] = play_stat['HTFormPtsStr'].apply(get_5game_ls)

play_stat['ATWinStreak3'] = play_stat['ATFormPtsStr'].apply(get_3game_ws)
play_stat['ATWinStreak5'] = play_stat['ATFormPtsStr'].apply(get_5game_ws)
play_stat['ATLossStreak3'] = play_stat['ATFormPtsStr'].apply(get_3game_ls)
play_stat['ATLossStreak5'] = play_stat['ATFormPtsStr'].apply(get_5game_ls)

play_stat.keys()

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS',
       'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5', 'AM1',
       'AM2', 'AM3', 'AM4', 'AM5', 'MW', 'HTFormPtsStr', 'ATFormPtsStr',
       'HTFormPts', 'ATFormPts', 'HTWinStreak3', 'HTWinStreak5',
       'HTLossStreak3', 'HTLossStreak5', 'ATWinStreak3', 'ATWinStreak5',
       'ATLossStreak3', 'ATLossStreak5'],
      dtype='object')

In [12]:
play_stat['HTGD'] = play_stat['HTGS'] - play_stat['HTGC']
play_stat['ATGD'] = play_stat['ATGS'] - play_stat['ATGC']

# Diff in points
play_stat['DiffPts'] = play_stat['HTP'] - play_stat['ATP']
play_stat['DiffFormPts'] = play_stat['HTFormPts'] - play_stat['ATFormPts']

# Diff in last year positions
# play_stat['DiffLP'] = play_stat['HomeTeamLP'] - play_stat['AwayTeamLP']

In [13]:
cols = ['HTGD','ATGD','DiffPts','DiffFormPts','HTP','ATP']
play_stat.MW = play_stat.MW.astype(float)

for col in cols:
    play_stat[col] = play_stat[col] / play_stat.MW

In [14]:
def only_hw(string):
    if string == 'H':
        return 'H'
    else:
        return 'NH'
    
play_stat['FTR'] = play_stat.FTR.apply(only_hw)
# Testing set (2018-19 season)
play_stat_test = play_stat[4940:]

In [15]:
play_stat.to_csv(data_dir + "final_dataset.csv")
play_stat_test.to_csv(data_dir + "test.csv")